In [ ]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as st
import time
from sklearn.decomposition import PCA
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar import var_model

Stock and Watson (2002) argue that factor models are either able to cope with breaks in the factor loadings in a fraction of the series, or can account for moderate parameter drift in all of the series. However, in empirical applications parameters may change dramatically due to important economic events. There may also be more gradual but nevertheless fundamental changes in economic structures that may have led to significant changes in the comovements of variables.

Thus, time-varying factor loadings lead not only to inconsistent estimates of the loadings but also to a larger dimension of the factor space.

# EDA: Macroeconomic Database FRED-MD

In [ ]:
# macroeconomic database of monthly U.S. indicators 
# such as output and income, the labor market and prices from 1959 to 2020
df = pd.read_csv(r"current.csv")
# according to the FRED-MD paper, these 7 factors have the greatest explanatory power, so we will use them from now on
fred7 = df[['USGOOD', 'PAYEMS', 'MANEMP', 'IPMANSICS', 'DMANEMP', 'INDPRO', 'CUMFNS']].copy()
# drop any column that has a nan value, as well as the first "transformation" row
fred7 = fred7.dropna()[1:]
# it turns out coronavirus is causing a huge number of outliers in our data, so we will drop the months after march 2020
fred7 = fred7[:-8]
# grab the dates for plotting purposes
dates = df['sasdate'][1:-9]

In [ ]:
# transform it into an numpy ndarray
fred7 = fred7.to_numpy().T
fred7

### Adjust for the non-stationarity in the dataset

In [ ]:
# difference the non-stationary series to stationarity
# check for non-stationary and take differences of non-stationary series
def non_stationary(timeseries):
    # Perform augmented Dickey-Fuller test and return whether the test statistic is greater than the critical value,
    # which means the time series is non-stationary. We pick an alpha = 0.10
    dftest = st.adfuller(timeseries, autolag='AIC')
    test_statistic = dftest[0]
    crit_value = dftest[4]['10%']
    return test_statistic > crit_value

stationary_fred7 = fred7.copy()
# print(len(stationary_fred7))
# print(len(stationary_fred7[0]))
for i in range(len(stationary_fred7)):
    if non_stationary(stationary_fred7[i]):
        diffs = []
        for j in range(1, len(stationary_fred7[i])):
            diffs.append(stationary_fred7[i][j] - stationary_fred7[i][j-1])
        diffs.insert(0, np.mean(diffs))
        stationary_fred7[i] = diffs

# print(stationary_fred7.shape)

### Standardize the dataset

In [ ]:
def center(series):
    zero_mean = series - np.mean(series)
    unit_variance = zero_mean / (np.std(series))
    return unit_variance

In [ ]:
# standardize all columns to have zero mean and unit variance
whitened_fred7 = np.empty(shape=(0,0))
for col in stationary_fred7:
    # don't apply this to malformed columns, i.e. columns with nan or non-floats
    if not isinstance(col[0], str):
        whitened_fred7 = np.append(whitened_fred7, center(col))
whitened_fred7 = whitened_fred7.reshape(stationary_fred7.shape)


In [ ]:
# plt.plot([np.log(i) for i in fred7[1]])
# plt.plot([np.log(i) for i in fred7[2]])
# plt.plot([np.log(i) for i in fred7[3]])
plt.plot(whitened_fred7[0])
plt.plot(whitened_fred7[1])
plt.plot(whitened_fred7[2])
plt.plot(whitened_fred7[3])
plt.plot(whitened_fred7[4])
plt.plot(whitened_fred7[5])
plt.plot(whitened_fred7[6])
# plt.legend(whitened_fred7[0], ['USGOOD', 'PAYEMS', 'MANEMP', 'IPMANSICS', 'DMANEMP', 'INDPRO', 'CUMFNS'])

In [ ]:
def plotter(index):
    fig, axs = plt.subplots(3)
    fig.figsize = [6.4, 9.6]
    axs[0].plot(fred7[index])
    axs[1].plot(stationary_fred7[index])
    axs[2].plot(whitened_fred7[index])
plotter(1)

In [ ]:
feat_cols = ['feature'+str(i) for i in range(whitened_fred7.shape[0])]
normalized_fred7 = pd.DataFrame(whitened_fred7.T,columns=feat_cols)
normalized_fred7

This is the normalized aggregate data we will use to test for time-varying loadings.

## Traditional PCA

In [ ]:
# Extract all principle components of the aggregate macro data
N = 6
pca_fred = PCA(n_components = N)
principalComponents_fred7 = pca_fred.fit_transform(normalized_fred7)
principalComponents_fred7

In [ ]:
principal_fred_df = pd.DataFrame(data = principalComponents_fred7
             , columns = ['PC'+str(i) for i in range(1, N+1)])
principal_fred_df

In [ ]:
fred_explained_variance = pca_fred.explained_variance_ratio_
fred_explained_variance_df = pd.DataFrame(fred_explained_variance, columns = ['Explained Variance'])
fred_explained_variance_df.rename(index = lambda x: 'PC'+str(x+1), inplace=True)
fred_explained_variance_df

In [ ]:
plt.plot(np.cumsum(fred_explained_variance))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

## Testing for time-varying loadings

Test the null hypothesis that the time loadings are time-invariant by testing the autocorrelation in the residuals.

In [ ]:
# Determine lag order from the average of all features
N = len(normalized_fred7.columns)
T = len(normalized_fred7.index)
optimal_lag = np.array([])
for j in range(0, N):
    k = 0
    highestCorr = 0
    for i in range(1, 10):
        cor = pd.Series.autocorr(normalized_fred7.iloc[:, j].astype(float), lag = i)
        if(cor > highestCorr):
            highestCorr = cor
            k = i
    optimal_lag = np.append(optimal_lag, k)
optimal_lag

In [ ]:
p = int(np.round(np.mean(optimal_lag)))
p

Find the rejection rate of the null hypothesis that the autocorrelations of the residual time series are not different from zero (also called the Ljung–Box test), thus testing for serial correlation between the features.

Plot all the features.

In [ ]:
test_fred_10 = normalized_fred7
model = VAR(test_fred_10.astype(float))
results_10 = model.fit(p, method = "ols")
results_10.plot();

In [ ]:
resid_test_10 = var_model.VARResults.test_whiteness(results_10, nlags=10, signif=0.05, adjusted=False)
resid_test_10.summary()

In [ ]:
resid_test_10.pvalue <= 0.05

Since $p \leq 0.05$, we reject the null hypothesis that the residual autocorrelations are 0. In other words, since the autocorrelations are significantly different from 0, the data is serially correlated, which shows seasonality.

In [ ]:
rejection_10 = np.array([])
for j in range(1, int(N/n)):
    test_fred = normalized_fred7
    model = VAR(test_fred.astype(float))
    results = model.fit(p, method = "ols")
    resid_test = var_model.VARResults.test_whiteness(results, nlags=250, signif=0.05, adjusted=False)
    rejection_10 = np.append(rejection_10, resid_test.pvalue)
rejection_10

rejection_CDF = pd.Series([var_model.VARResults.test_whiteness(results, nlags=i, signif=0.05, adjusted=False).pvalue for i in range(200, 300)])
rejection_PDF = rejection_CDF - rejection_CDF.shift(2)
# var_model.VARResults.test_whiteness?
# plt.plot(range(200, 300), rejection_CDF)
plt.plot(range(200, 300), rejection_PDF)

In [ ]:
rejection_rate_10 = rejection_10.sum()/len(rejection_10)
print(str(rejection_rate_10*100)+'%')

In [ ]:
rejection_rate = np.array([])
for i in np.array([10, 20, 50]):
    n = i # number of variable-specific tests
    j_index = 0
    rejection_i = np.array([])
    for j in range(1, int(N/i)):
        test_fred = normalized_fred7.iloc[:, np.arange(j_index, j_index+n)]
        model = VAR(test_fred.astype(float))
        results = model.fit(p, method = "ols")
        resid_test = var_model.VARResults.test_whiteness(results, nlags=10, signif=0.05, adjusted=False)
        rejection_i = np.append(rejection_i, resid_test.pvalue <= 0.05)
        j_index += n
    rejection_rate_i = rejection_i.sum()/len(rejection_i)
    rejection_rate = np.append(rejection_rate, str(rejection_rate_10*100)+'%')
rejection_rate

In [ ]:
rejection_rate_df = pd.DataFrame(rejection_rate, columns = ['Reject'])
rejection_rate_df = rejection_rate_df.rename(index = lambda x: 'N = '+ str(np.array([10, 20, 50]).item(x)))
rejection_rate_df